In [330]:
import requests
import pandas as pd
from datetime import date


In [331]:
# static page scrapping with Selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

def get_rendered_html():
    """
    This function scrap data from dynamic webpage by rendering it 
    and return page source code for next processing
    """
    url = "https://zelezna.sk/ponuka-bytov/"

    options = Options()
    options.headless = True
    options.add_argument("--window-size=1920,1200")
    driver = webdriver.Chrome("C:/Users/tomas/.wdm/drivers/chromedriver/win32/96.0.4664.45/chromedriver.exe")
    driver.get(url) # open url in browser
    html_page = driver.page_source # get rendered page source

    driver.quit() # close browser
    return html_page


In [332]:
def get_listings(html_page):
    """
    This function gets all property listings on homepage
    and return list of urls of all listed properties 
    """

    listings = "elastic-portfolio-item"
    html_soup = BeautifulSoup(html_page, "html.parser") # parse html sources
    flat_listings =  html_soup.find_all("div",listings) # find all "listings class"

    listings_url = []
    # loop trough listings and get listing url
    for flat in flat_listings:
        item = flat.find("a")
        url = item.get("href") 
        listings_url.append(url)
    
    return listings_url

In [337]:
def get_flat_data(listings_url):
    """
    This function collect data for any single property url
    in listings_url list,clean and saves it to csv file and return DataFrame
    """
    property_list = []

    for l in listings_url:

        property_url = l

        r = requests.get(property_url)
        page_source = r.content
        my_soup = BeautifulSoup(page_source, "html.parser")

        # Content CSS classes
        project_title = "row project-title"
        title_2 = "vc_column-inner"
        features_class = "nectar-fancy-ul"
        info_class = "wpb_text_column"


        my_dict = dict()

        # get id 
        project_title = my_soup.find("div",title_2)
        today = date.today()
        my_dict["updateDate"] = today.strftime("%d-%m-%Y")
        try:
            my_dict["flatNumber"] = project_title.h2.text 
        
        except:
            my_dict["flatNumber"] = project_title.h1.text 
            
        # get features, clean and save to dict
        project_features = my_soup.find("div",features_class)
        features = project_features.find_all("li")
        my_dict["roomsNumber"] = int(features[0].text.split(":")[1].replace(" ",""))
        floors_number = str(features[1].text.split(":")[1].replace(" ",""))
        my_dict["floorsNumber"] = int(floors_number.split(".")[0].strip())
        my_dict["flatArea"] = float(features[2].text.split(":")[1].replace(" ","").replace(",","."))
        my_dict["balconyTerraceArea"] = (None if features[3].text.split(":")[1] == "\xa0–" else float(features[3].text.split(":")[1].replace(" ","").replace(",",".")))
        my_dict["totalArea"] = float(features[4].text.split(":")[1].replace(" ","").replace(",","."))
        
        #get status and price for property
        sales_data = my_soup.find_all("div",info_class)
        my_dict["salesPrice"] = (None if len(sales_data[2].h3.text.split(":")[1].replace("€","").strip()) <=1 else float(sales_data[2].h3.text.split(":")[1].replace("\n","").replace("€","").replace(",",".").replace(" ","").strip()))
        my_dict["propStatus"] = sales_data[1].h3.text.split(":")[1].replace("\n","").strip()

        property_list.append(my_dict)

    df = pd.DataFrame.from_dict(property_list)
    df = df.set_index("updateDate")
    df.to_csv("zelezna_studenka.csv")
    return df

In [338]:
#flat_listings
html_page = get_rendered_html()
listings_url = get_listings(html_page)
properties = get_flat_data(listings_url)
properties.head()

C:\Users\tomas\AppData\Local\Temp/ipykernel_22152/503647780.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:/Users/tomas/.wdm/drivers/chromedriver/win32/96.0.4664.45/chromedriver.exe")


,flatNumber,roomsNumber,floorsNumber,flatArea,balconyTerraceArea,totalArea,salesPrice,propStatus
updateDate,,,,,,,,
28-11-2021,C 12 Apartmán C5.1,4,5,196.73,200.68,397.41,1067082.0,Voľný
28-11-2021,A 9 Apartmán A5.1,4,5,133.88,200.90,334.78,803376.0,Voľný
28-11-2021,C 11 Apartmán C4.3,3,4,103.44,14.88,118.32,NaN,Predaný
28-11-2021,C 10 Apartmán C4.2,2,4,48.60,17.19,65.79,NaN,Predaný
28-11-2021,C 9 Apartmán C4.1,3,4,90.03,7.93,97.96,377454.0,Voľný


In [335]:
import requests
import pandas as pd
from datetime import date

def get_flat_data(listings_url):
    property_list = []

    for l in listings_url:

        property_url = l

        r = requests.get(property_url)
        page_source = r.content
        my_soup = BeautifulSoup(page_source, "html.parser")

        # Content CSS classes
        project_title = "row project-title"
        title_2 = "vc_column-inner"
        features_class = "nectar-fancy-ul"
        info_class = "wpb_text_column"


        my_dict = dict()

        # get id 
        project_title = my_soup.find("div",title_2)
        today = date.today()
        my_dict["updateDate"] = today.strftime("%d-%m-%Y")
        try:
            my_dict["flatNumber"] = project_title.h2.text 
        
        except:
            my_dict["flatNumber"] = project_title.h1.text 
            
        # get features, clean and save to dict
        project_features = my_soup.find("div",features_class)
        features = project_features.find_all("li")
        my_dict["roomsNumber"] = int(features[0].text.split(":")[1].replace(" ",""))
        floors_number = str(features[1].text.split(":")[1].replace(" ",""))
        my_dict["floorsNumber"] = int(floors_number.split(".")[0].strip())
        my_dict["flatArea"] = float(features[2].text.split(":")[1].replace(" ","").replace(",","."))
        my_dict["balconyTerraceArea"] = (None if features[3].text.split(":")[1] == "\xa0–" else float(features[3].text.split(":")[1].replace(" ","").replace(",",".")))
        my_dict["totalArea"] = float(features[4].text.split(":")[1].replace(" ","").replace(",","."))
        
        #get status and price for property
        sales_data = my_soup.find_all("div",info_class)
        my_dict["salesPrice"] = (None if len(sales_data[2].h3.text.split(":")[1].replace("€","").strip()) <=1 else float(sales_data[2].h3.text.split(":")[1].replace("\n","").replace("€","").replace(",",".").replace(" ","").strip()))
        my_dict["propStatus"] = str(sales_data[1].h3.text.split(":")[1].replace("\n","").strip())

        property_list.append(my_dict)

    df = pd.DataFrame.from_dict(property_list)
    df = df.set_index("updateDate")
    df.to_csv("zelezna_studenka.csv")
    return df



In [336]:
df_col = ['flatNumber', 'roomsNumber', 'floorsNumber', 'flatArea',
       'balconyTerraceArea', 'totalArea', 'salesPrice', 'propStatus']
df

,flatNumber,roomsNumber,floorsNumber,flatArea,balconyTerraceArea,totalArea,salesPrice,propStatus
updateDate,,,,,,,,
28-11-2021,C 12 Apartmán C5.1,4,5.NP,196.73,200.68,397.41,1067082.0,Voľný
28-11-2021,A 9 Apartmán A5.1,4,5.NP,133.88,200.90,334.78,803376.0,Voľný
28-11-2021,C 11 Apartmán C4.3,3,4.NP,103.44,14.88,118.32,NaN,Predaný
28-11-2021,C 10 Apartmán C4.2,2,4.NP,48.60,17.19,65.79,NaN,Predaný
28-11-2021,C 9 Apartmán C4.1,3,4.NP,90.03,7.93,97.96,377454.0,Voľný
28-11-2021,B 12 Byt B4.3,2,4.NP,56.56,15.29,71.85,NaN,Predaný
28-11-2021,B 11 Byt B4.2,4,4.NP,127.59,26.31,154.00,575110.8,Voľný
28-11-2021,B 10 Byt B4.1,2,4.NP,56.05,14.39,70.44,NaN,Predaný
28-11-2021,A 8 Apartmán A4.2,3,4.NP,94.02,7.00,101.02,390799.2,Voľný


In [310]:
prices = []
for l in listings_url:

    property_url = l

    r = requests.get(property_url)
    page_source = r.content
    my_soup = BeautifulSoup(page_source, "html.parser")

    # Content CSS classes
    info_class = "wpb_text_column"
    #get status and price for property
    sales_data = my_soup.find_all("div",info_class)

    data = (None if len(sales_data[2].h3.text.split(":")[1].replace("€","").strip()) <=1 else float(sales_data[2].h3.text.split(":")[1].replace("\n","").replace("€","").replace(",",".").replace(" ","").strip()))
    prices.append(data)
    

In [311]:
prices

[1067082.0,
 803376.0,
 None,
 None,
 377454.0,
 None,
 575110.8,
 None,
 390799.2,
 458013.6,
 448905.6,
 None,
 None,
 272556.0,
 577803.6,
 270127.2,
 390799.2,
 458013.6,
 449103.6,
 None,
 461934.0,
 272556.0,
 None,
 270127.2,
 None,
 458013.6,
 None,
 410373.6,
 301123.2,
 548618.4,
 164346.0,
 438064.8,
 414981.6]

In [260]:
df

,updateDate,flatNumber,roomsNumber,floorsNumber,flatArea,balconyTerraceArea,totalArea,propStatus
0,28-11-2021,C12 Apartmán C5.1,4,5.NP,196.73,200.68,397.41,Voľný
1,28-11-2021,A9 Apartmán A5.1,4,5.NP,133.88,200.90,334.78,Voľný
2,28-11-2021,C11 Apartmán C4.3 (predaný),3,4.NP,103.44,14.88,118.32,Predaný
3,28-11-2021,C10 Apartmán C4.2 (predaný),2,4.NP,48.60,17.19,65.79,Predaný
4,28-11-2021,C9 Apartmán C4.1,3,4.NP,90.03,7.93,97.96,Voľný
5,28-11-2021,B12 Byt B4.3 (predaný),2,4.NP,56.56,15.29,71.85,Predaný
6,28-11-2021,B11 Byt B4.2,4,4.NP,127.59,26.31,154.00,Voľný
7,28-11-2021,B10 Byt B4.1 (predaný),2,4.NP,56.05,14.39,70.44,Predaný
8,28-11-2021,A8 Apartmán A4.2,3,4.NP,94.02,7.00,101.02,Voľný
9,28-11-2021,A7 Apartmán A4.1,3,4.NP,102.47,19.80,122.26,Voľný


In [237]:
my_dict

{'id': 'A2 Apartmán A1.2',
 'Počet pokoju': 3,
 'Podlaží': '1.NP',
 'Plocha bytu': 96.94,
 'Balkón/terasa': 11.78,
 'Celková plocha': 108.72,
 'SalesPrice': None}

In [242]:
cc = (None if sales_staus[2].h3.text.split(":")[1].replace("\n","") == "-" else float(sales_staus[2].h3.text.split(":")[1].replace("\n","").replace("€","").replace(",",".").replace(" ","").strip()))
print(cc)

ValueError: could not convert string to float: '–'

In [246]:
sales_staus = my_soup.find_all("div",info_class)

sales_staus[1].h3.text.split(":")[1].replace("\n","").strip()
#float(sales_staus[2].h3.text.split(":")[1].replace("\n","").replace("€","").replace(",",".").replace(" ","").strip())

'Predaný'

In [198]:
len(property_list)

33

In [169]:
features[1].text.split(":")[1].replace(" ","")

'5.NP'

In [194]:
y = (None if features[3].text.split(":")[1] == "\xa0–" else float(features[3].text.split(":")[1].replace(" ","").replace(",",".")))
print(y)

None


In [182]:
if features[3].text.split(":")[1] == "-" None  

'\xa0–'

In [179]:
project_features = my_soup.find("div",features_class)
features = project_features.find_all("li")
for i in features:
    print(i.text)

Počet izieb: 1
Podlažie: 1. NP
Plocha bytu (m²): 39,13
Balkón/terasa (m²): –
Celková plocha (m2): 39,13
